In [101]:
import pandas as pd
import numpy as np
import pymssql
import glob
import os

In [102]:
files = glob.glob('*.xlsx')
print(files)

['выдачи по партнерам.xlsx']


In [103]:
  df = pd.read_excel(files[0])

In [104]:
df.head()

,dtEvent,eventName,idCampaign,idCampaignTitle,idPartner,idPartnerAttrTitle,idPartnerTitle,params.name,params.value,customUserId,idDevice,idProfile
0,2024-06-16,Выдача займа,12844791,AQ_amg,10880,AMG,AMG,"['Sum','Type']","['18950','Новый']",4530792.0,47642548659,113542310468
1,2024-06-16,Выдача займа,16906547,Upsmobi_MobX_Android,10612,UpsMobi,UpsMobi,"['Sum','Type']","['13000','Повторный']",4241028.0,3336480990,113721595608
2,2024-06-15,Выдача займа,11301543,MS_Vf_AOS_RU,10395,Vfineads,Vfineads,"['Sum','Type']","['8790','Повторный']",2561452.0,40852998169,111443791693
3,2024-06-16,Выдача займа,11693074,MS_AMG_AOS_RU,10880,AMG,AMG,"['Sum','Type']","['2000','Повторный']",3708944.0,84792820525,113686337549
4,2024-06-16,Выдача займа,11693074,MS_AMG_AOS_RU,10880,AMG,AMG,"['Sum','Type']","['23000','Новый']",4531768.0,85714255061,113661280693


In [105]:
filtered_df = df[df['params.value'].apply(lambda x: 'Новый' in x)]

In [106]:
filtered_df.head()

,dtEvent,eventName,idCampaign,idCampaignTitle,idPartner,idPartnerAttrTitle,idPartnerTitle,params.name,params.value,customUserId,idDevice,idProfile
0,2024-06-16,Выдача займа,12844791,AQ_amg,10880,AMG,AMG,"['Sum','Type']","['18950','Новый']",4530792.0,47642548659,113542310468
4,2024-06-16,Выдача займа,11693074,MS_AMG_AOS_RU,10880,AMG,AMG,"['Sum','Type']","['23000','Новый']",4531768.0,85714255061,113661280693
8,2024-06-15,Выдача займа,17967796,Dobroloan_appska_iguazu,11741,Iguazu for Dobrozaim,Iguazu for Dobrozaim,"['Sum','Type']","['37000','Новый']",4530138.0,69290265047,113453564049
23,2024-06-10,Выдача займа,11874366,Joinship_MobX_Android,10778,JoinShip,JoinShip,"['Sum','Type']","['3000','Новый']",4485159.0,5437109182,108362789459
28,2024-06-11,Выдача займа,11884758,Admile_MobX_Android,10687,Admile,Admile,"['Sum','Type']","['10000','Новый']",4507947.0,58241376693,110658557030


In [107]:
len(filtered_df)

398

In [108]:
# сохраняю отфильтрованный исходник
filtered_df.to_excel('1.xlsx', index=False)

In [109]:
filtered_df['customUserId'].dtype

dtype('float64')

In [110]:
df_for_sql = filtered_df[['dtEvent','customUserId']].reset_index(drop=True)

In [111]:
df_for_sql.head()

,dtEvent,customUserId
0,2024-06-16,4530792.0
1,2024-06-16,4531768.0
2,2024-06-15,4530138.0
3,2024-06-10,4485159.0
4,2024-06-11,4507947.0


In [112]:
df_for_sql['customUserId'] = df_for_sql['customUserId'].astype('int')

In [113]:
df_for_sql.head()

,dtEvent,customUserId
0,2024-06-16,4530792
1,2024-06-16,4531768
2,2024-06-15,4530138
3,2024-06-10,4485159
4,2024-06-11,4507947


In [ ]:
conn = pymssql.connect(server="SERVER")
cursor = conn.cursor()

In [115]:
# Удаление старых данных из таблицы


In [ ]:
cursor.execute(f"""DELETE FROM TABL;""")
conn.commit()

In [ ]:
sql = f"""
INSERT INTO TABL (dtEvent, customUserId) VALUES (%s, %s)
"""

In [118]:
sql_data = tuple(map(tuple, df_for_sql.values))

In [119]:
# sql_data

In [120]:
cursor.executemany(sql, sql_data)
conn.commit()   # Сохранение изменений

In [ ]:
# Выполнение дополнительного SQL-запроса
additional_sql = """
SELECT p.[dtEvent]
      ,p.[customUserId]
      ,q.keyDZ
      ,zi.NameMFO as 'Источник трафика'
      ,zi.NameKeySite as 'Канал где была оставлена заявка'
      ,q.NameToHome as 'Канал выдачи'
      ,CASE when q.IsNewCli = -1 Then 'New' ELSE 'Regular' END as 'Тип клиента'
      ,cast(q.dat as date) as 'Дата выдачи'
  FROM TABL p
      CROSS APPLY (
          SELECT TOP (1)
              dat,
              keypart,
              keydz,
              sumcred,
              IsNewCli,
              NameToHome,
              keyIZ
          FROM TABL_dog
          WHERE keypart = p.[customUserId] AND cast(dat as date) <= CAST(p.dtEvent as date)
      ) AS q
  LEFT JOIN tabl_lidy zi ON zi.keyIZ = q.keyIZ
"""

In [122]:
cursor.execute(additional_sql)

In [123]:
# Если вы хотите сохранить результаты запроса в DataFrame:
results = cursor.fetchall()
columns = [column[0] for column in cursor.description]
result_df = pd.DataFrame(results, columns=columns)

In [124]:
cursor.close()
conn.close()

In [125]:
result_df.head()

,dtEvent,customUserId,keyDZ,Источник трафика,Канал где была оставлена заявка,Канал выдачи,Тип клиента,Дата выдачи
0,2024-06-14,4529212.0,11655845,leadgid,Онлайн в ЛК,Выдача по телефону,New,2024-06-14
1,2024-06-16,4532843.0,11664721,Сайт Доброзайм,Мобильное приложение,Через ЛК,New,2024-06-16
2,2024-06-11,4523940.0,11643017,leadgid,Онлайн в ЛК,Выдача по телефону,New,2024-06-11
3,2024-06-16,4532331.0,11663528,Сайт Доброзайм,Мобильное приложение,Мобильное приложение,New,2024-06-16
4,2024-06-15,4531489.0,11661503,Сайт Доброзайм,Мобильное приложение,Мобильное приложение,New,2024-06-15


In [126]:
result_df.to_excel('query_results.xlsx', index=False)

In [127]:
# Загрузка данных из файлов Excel
df1 = pd.read_excel('1.xlsx')
df2 = pd.read_excel('query_results.xlsx')

In [128]:
df2.head()

,dtEvent,customUserId,keyDZ,Источник трафика,Канал где была оставлена заявка,Канал выдачи,Тип клиента,Дата выдачи
0,2024-06-14,4529212,11655845,leadgid,Онлайн в ЛК,Выдача по телефону,New,2024-06-14
1,2024-06-16,4532843,11664721,Сайт Доброзайм,Мобильное приложение,Через ЛК,New,2024-06-16
2,2024-06-11,4523940,11643017,leadgid,Онлайн в ЛК,Выдача по телефону,New,2024-06-11
3,2024-06-16,4532331,11663528,Сайт Доброзайм,Мобильное приложение,Мобильное приложение,New,2024-06-16
4,2024-06-15,4531489,11661503,Сайт Доброзайм,Мобильное приложение,Мобильное приложение,New,2024-06-15


In [129]:
# Объединение DataFrame по ключу 'customUserId'
merged_df = pd.merge(df1, df2, on='customUserId', how='left')

In [130]:
merged_df.head()

,dtEvent_x,eventName,idCampaign,idCampaignTitle,idPartner,idPartnerAttrTitle,idPartnerTitle,params.name,params.value,customUserId,idDevice,idProfile,dtEvent_y,keyDZ,Источник трафика,Канал где была оставлена заявка,Канал выдачи,Тип клиента,Дата выдачи
0,2024-06-16,Выдача займа,12844791,AQ_amg,10880,AMG,AMG,"['Sum','Type']","['18950','Новый']",4530792,47642548659,113542310468,2024-06-16,11659873,Сайт Доброзайм,Мобильное приложение,Мобильное приложение,New,2024-06-16
1,2024-06-16,Выдача займа,11693074,MS_AMG_AOS_RU,10880,AMG,AMG,"['Sum','Type']","['23000','Новый']",4531768,85714255061,113661280693,2024-06-16,11662217,Сайт Доброзайм,Мобильное приложение,Мобильное приложение,New,2024-06-16
2,2024-06-15,Выдача займа,17967796,Dobroloan_appska_iguazu,11741,Iguazu for Dobrozaim,Iguazu for Dobrozaim,"['Sum','Type']","['37000','Новый']",4530138,69290265047,113453564049,2024-06-15,11658248,Sravni.ru,Онлайн в ЛК,Мобильное приложение,New,2024-06-15
3,2024-06-10,Выдача займа,11874366,Joinship_MobX_Android,10778,JoinShip,JoinShip,"['Sum','Type']","['3000','Новый']",4485159,5437109182,108362789459,2024-06-10,11555224,Сайт Доброзайм,Мобильное приложение,Мобильное приложение,New,2024-06-10
4,2024-06-11,Выдача займа,11884758,Admile_MobX_Android,10687,Admile,Admile,"['Sum','Type']","['10000','Новый']",4507947,58241376693,110658557030,2024-06-11,11642307,Leadstech (Котозайм),Онлайн в ЛК,Мобильное приложение,New,2024-06-11


In [131]:
merged_df = merged_df.drop(columns='dtEvent_y')

In [132]:
merged_df = merged_df.rename(columns={'dtEvent_x':'dtEvent'})

In [133]:
merged_df.head()

,dtEvent,eventName,idCampaign,idCampaignTitle,idPartner,idPartnerAttrTitle,idPartnerTitle,params.name,params.value,customUserId,idDevice,idProfile,keyDZ,Источник трафика,Канал где была оставлена заявка,Канал выдачи,Тип клиента,Дата выдачи
0,2024-06-16,Выдача займа,12844791,AQ_amg,10880,AMG,AMG,"['Sum','Type']","['18950','Новый']",4530792,47642548659,113542310468,11659873,Сайт Доброзайм,Мобильное приложение,Мобильное приложение,New,2024-06-16
1,2024-06-16,Выдача займа,11693074,MS_AMG_AOS_RU,10880,AMG,AMG,"['Sum','Type']","['23000','Новый']",4531768,85714255061,113661280693,11662217,Сайт Доброзайм,Мобильное приложение,Мобильное приложение,New,2024-06-16
2,2024-06-15,Выдача займа,17967796,Dobroloan_appska_iguazu,11741,Iguazu for Dobrozaim,Iguazu for Dobrozaim,"['Sum','Type']","['37000','Новый']",4530138,69290265047,113453564049,11658248,Sravni.ru,Онлайн в ЛК,Мобильное приложение,New,2024-06-15
3,2024-06-10,Выдача займа,11874366,Joinship_MobX_Android,10778,JoinShip,JoinShip,"['Sum','Type']","['3000','Новый']",4485159,5437109182,108362789459,11555224,Сайт Доброзайм,Мобильное приложение,Мобильное приложение,New,2024-06-10
4,2024-06-11,Выдача займа,11884758,Admile_MobX_Android,10687,Admile,Admile,"['Sum','Type']","['10000','Новый']",4507947,58241376693,110658557030,11642307,Leadstech (Котозайм),Онлайн в ЛК,Мобильное приложение,New,2024-06-11


In [134]:
# Переименование значений в столбце "Тип клиента"
merged_df['Тип клиента'] = merged_df['Тип клиента'].replace({'New': 'Новый', 'Regular': 'Постоянный'})

In [135]:
merged_df.head()

,dtEvent,eventName,idCampaign,idCampaignTitle,idPartner,idPartnerAttrTitle,idPartnerTitle,params.name,params.value,customUserId,idDevice,idProfile,keyDZ,Источник трафика,Канал где была оставлена заявка,Канал выдачи,Тип клиента,Дата выдачи
0,2024-06-16,Выдача займа,12844791,AQ_amg,10880,AMG,AMG,"['Sum','Type']","['18950','Новый']",4530792,47642548659,113542310468,11659873,Сайт Доброзайм,Мобильное приложение,Мобильное приложение,Новый,2024-06-16
1,2024-06-16,Выдача займа,11693074,MS_AMG_AOS_RU,10880,AMG,AMG,"['Sum','Type']","['23000','Новый']",4531768,85714255061,113661280693,11662217,Сайт Доброзайм,Мобильное приложение,Мобильное приложение,Новый,2024-06-16
2,2024-06-15,Выдача займа,17967796,Dobroloan_appska_iguazu,11741,Iguazu for Dobrozaim,Iguazu for Dobrozaim,"['Sum','Type']","['37000','Новый']",4530138,69290265047,113453564049,11658248,Sravni.ru,Онлайн в ЛК,Мобильное приложение,Новый,2024-06-15
3,2024-06-10,Выдача займа,11874366,Joinship_MobX_Android,10778,JoinShip,JoinShip,"['Sum','Type']","['3000','Новый']",4485159,5437109182,108362789459,11555224,Сайт Доброзайм,Мобильное приложение,Мобильное приложение,Новый,2024-06-10
4,2024-06-11,Выдача займа,11884758,Admile_MobX_Android,10687,Admile,Admile,"['Sum','Type']","['10000','Новый']",4507947,58241376693,110658557030,11642307,Leadstech (Котозайм),Онлайн в ЛК,Мобильное приложение,Новый,2024-06-11


In [136]:
# Замена пустых значений в столбце "Канал где была оставлена заявка" на "Самовывоз"
merged_df['Канал где была оставлена заявка'] = merged_df['Канал где была оставлена заявка'].fillna('Самовывоз')

In [137]:
# Сохранение объединенного DataFrame в новый файл Excel
merged_df.to_excel('Готовый.xlsx', index=False)

In [138]:
# Удаление ненужных файлов
os.remove('1.xlsx')
os.remove('query_results.xlsx')